# Example of use of functions.py script
17/08/2021   
Gabriel Besombes   
#     

## Import needed packages
---

`sys` is needed to add the path to the functions.py file to the path variable

In [1]:
import sys
import opensilexClientToolsPython
import json

Insert the new path at position 1. Position 0 is the script path (or '' in REPL)    
Change "C:\\Users\\besombes\\Documents\\gitlab\\opensilex-ws-python-client" to the corresponding path to the cloned repository on your computer.

In [2]:
sys.path.insert(1, "C:\\Users\\besombes\\Documents\\gitlab\\opensilex-ws-python-client")

The file can now be imported

In [4]:
import functions

___
#   

## Connect to opensilex
___

__WARNING__ : You need to have a JSON file with your credentials (for me it's credentials.json and contains the following :   
`{
    "identifier":"guest@opensilex.org",
    "password":"guest_password",
    "host":"server_ip:server_port/rest"
}`)   
or to pass your credentials directly to `pythonClient.connect_to_opensilex_ws()`

In [5]:
with open('credentials.json') as f:
    credentials_dict = json.load(f)


In [7]:
pythonClient = opensilexClientToolsPython.ApiClient()
pythonClient.connect_to_opensilex_ws(**credentials_dict)

___
#    

## Migrate Variables from csv
___

To migrate variables from a csv a schema is needed. Here the default schema is used :

In [8]:
variables_schema = {
    'trait':'trait.uri',
    'trait_name':'trait.label',
    'entity':{
        'name':'entity.label',
        'uri':'entity.uri',
        'description':'entity.comment'
    },
    'characteristic':{
        'name':'characteristic.label',
        'uri':'characteristic.uri',
        'description':'characteristic.comment'
    },
    'method':{
        'name':'method.label',
        'uri':'method.uri',
        'description':'method.comment'
    },
    'unit':{
        'name':'unit.label',
        'uri':'unit.uri',
        'description':'unit.comment',
    },
    'uri':'variable.uri',
    'name':'variable.label',
    'description':'variable.description',
    'datatype':'variable.datatype',
    'alternative_name':'variable.alternative_name',
    'time_interval':'variable.timeinterval',
    'sampling_interval':'variable.sampleinterval'
}

Trying to import example variables from `.\csv_example\extended_variables_testing.csv`   
__NOTE__ : This will most likely fail as these variables probably already exist

In [7]:
res_csv = functions.migrate_variables_from_csv(
    pythonClient=pythonClient, 
    csv_path="./csv_example/extended_variables_testing.csv",
    variables_schema=variables_schema
)
res_csv

2021-10-26 14:53:22,858 INFO         [functions.py:574] Update mode variable is set to False


/home/besombes/anaconda3/envs/PyR/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
2021-10-26 14:53:23,062 INFO         [functions.py:908] Object {'name': 'test_entity_name_Gabriel_1', 'uri': 'test:set/test_entity_uri_Gabriel_1', 'description': 'test_entity_comment_Gabriel_1'} at row 0 wasn't created as an object with that name already exists.
That object was skipped and will appear in the "already_existed.csv" file.
The object used instead is {'uri': 'test:set/test_entity_uri_Gabriel_1', 'name': 'test_entity_name_Gabriel_1', 'description': 'test_entity_comment_Gabriel_1', 'exact_match': [], 'close_match': [], 'broad_match': [], 'narro

,trait,trait_name,entity,characteristic,method,unit,uri,name,description,datatype,alternative_name,time_interval,sampling_interval,exact_match,close_match,broad_match,narrow_match


If you get an error the most likely cause is the format of your csv. There are two ways to solve this :
 * If you know how to use pandas you should try to open your csv with pandas and pass the created DataFrame to functions.migrate_variables().
 * If you don't know how to use pandas try to save your csv with comas "," as separators

If you want more details on the outputs you can check the debug.log file.

___
#    

## Migrate Variables from googlesheet
---

In [9]:
variables_schema = {
    'trait_name':'trait.label',
    'entity':{
        'name':'entity.label'
    },
    'characteristic':{
        'name':'characteristic.label'
    },
    'method':{
        'name':'method.label'
    },
    'unit':{
        'name':'unit.label',
        'alternative_symbol':'symbol_column'
    },
    'name':'variable.name',
    'datatype':'variable.datatype',
    'description':'variable.description'
}

In [9]:
res_sheet = functions.migrate_variables_from_googlesheet(
    pythonClient=pythonClient,
    spreadsheet_url='https://docs.google.com/spreadsheets/d/1mqmXkSekWhdde7Ic4baiw9R3jSkS90CwTuQbZoaUIVk',
    gid_number='794620992',
    variables_schema=variables_schema
)
res_sheet

2021-10-26 14:53:44,494 INFO         [functions.py:362] https://docs.google.com/spreadsheets/d/1mqmXkSekWhdde7Ic4baiw9R3jSkS90CwTuQbZoaUIVk/gviz/tq?tqx=out:csv&gid=794620992
2021-10-26 14:53:45,296 INFO         [functions.py:574] Update mode variable is set to False


/home/besombes/anaconda3/envs/PyR/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
2021-10-26 14:53:45,512 INFO         [functions.py:908] Object {'name': 'Entity test Google NEWSheet'} at row 0 wasn't created as an object with that name already exists.
That object was skipped and will appear in the "already_existed.csv" file.
The object used instead is {'uri': 'test:set/variables#variable.entity.entity-test-google-newsheet', 'name': 'Entity test Google NEWSheet', 

,trait,trait_name,entity,characteristic,method,unit,uri,name,description,datatype,alternative_name,time_interval,sampling_interval,exact_match,close_match,broad_match,narrow_match


---
#     

## Get variables from opensilex
___

Getting variables that have "Orge" in their uri or "Soja" in their name

In [10]:
res = functions.get_variables(
    pythonClient=pythonClient,
    uri_match = "Orge",
    name_match = "Soja"
)

In [11]:
res

,trait.uri,trait.label,entity.label,entity.uri,entity.comment,characteristic.label,characteristic.uri,characteristic.comment,method.label,method.uri,...,unit.label,unit.uri,unit.comment,variable.uri,variable.label,variable.description,variable.datatype,variable.alternative_name,variable.timeinterval,variable.sampleinterval
0,None,None,Plants population,test:entity/plants-population,A population of plants whose characteristics a...,Number of plants of 'Orge d'hiver',test:characteristic/number-of-Orge_d-hiver,Number of plants of Orge d'hiver in a population,Rangs,test:method/Rangs,...,Unitless,test:set/variables#variable.unit.unitless,None,test:variable/number-of-plants-Orge_d-hiver-Rangs,Number of plants of Orge d'hiver with Rangs me...,None,http://www.w3.org/2001/XMLSchema#integer,None,None,None
1,None,None,Plants population,test:entity/plants-population,A population of plants whose characteristics a...,Number of plants of 'Orge de printemps',test:characteristic/number-of-Orge_de_printemps,Number of plants of Orge de printemps in a pop...,Rangs,test:method/Rangs,...,Unitless,test:set/variables#variable.unit.unitless,None,test:variable/number-of-plants-Orge_de_printem...,Number of plants of Orge de printemps with Ran...,None,http://www.w3.org/2001/XMLSchema#integer,None,None,None
2,None,None,Plants population,test:entity/plants-population,A population of plants whose characteristics a...,Number of plants of 'Soja',test:characteristic/number-of-Soja,Number of plants of Soja in a population,Rangs,test:method/Rangs,...,Unitless,test:set/variables#variable.unit.unitless,None,test:variable/number-of-plants-Soja-Rangs,Number of plants of Soja with Rangs method,None,http://www.w3.org/2001/XMLSchema#integer,None,None,None


By default the variables are saved in "variables_export.csv"

___
#    

## Create User accounts from google sheet
___

In [10]:
functions.create_users_from_google_sheet(
    pythonClient = pythonClient, 
    sheet_id = "1z1vna9ppSxlLET8O0LqB7yD2N1cPJ2N0aZV3X1u2W8g",
    sheet_name  = "Users_examples",
    maping = {
        "uri": "orcid",
        "first_name": "first_name",
        "last_name": "family_name",
        "email": "email",
        "admin": "isadmin"
        })

2022-01-06 16:05:23,765 ERROR        [functions.py:1371] Exception : (409)
Reason: 
HTTP response headers: HTTPHeaderDict({'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'origin, content-type, accept, authorization', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS, HEAD', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/json', 'Content-Length': '347', 'Date': 'Thu, 06 Jan 2022 15:04:43 GMT'})
HTTP response body: {
  "metadata" : {
    "pagination" : {
      "pageSize" : 0,
      "currentPage" : 0,
      "totalCount" : 0,
      "totalPages" : 0
    },
    "status" : [ ],
    "datafiles" : [ ]
  },
  "result" : {
    "title" : "User already exists",
    "message" : "Duplicated email: example@opensilex.org",
    "stack" : null,
    "fullstack" : null
  }
}



In [11]:
functions.create_users_from_google_sheet(
    pythonClient = pythonClient, 
    sheet_id = "1hcWI9BoMlJLMi0RGfl1C2pDdnxEVDaUvbCY14VxUZ4g",
    sheet_name  = "DiaPhen_users",
    maping = {
        "uri": "orcid",
        "first_name": "first_name",
        "last_name": "family_name",
        "email": "email",
        "admin": "isadmin"
        })

2022-01-06 16:06:56,603 ERROR        [functions.py:1371] Exception : (409)
Reason: 
HTTP response headers: HTTPHeaderDict({'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'origin, content-type, accept, authorization', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS, HEAD', 'Vary': 'Accept-Encoding', 'Content-Type': 'application/json', 'Content-Length': '351', 'Date': 'Thu, 06 Jan 2022 15:06:16 GMT'})
HTTP response body: {
  "metadata" : {
    "pagination" : {
      "pageSize" : 0,
      "currentPage" : 0,
      "totalCount" : 0,
      "totalPages" : 0
    },
    "status" : [ ],
    "datafiles" : [ ]
  },
  "result" : {
    "title" : "User already exists",
    "message" : "Duplicated email: isabelle.sanchez@inrae.fr",
    "stack" : null,
    "fullstack" : null
  }
}

2022-01-06 16:06:56,670 ERROR        [functions.py:1371] Exception : (409)
Reason: 
HTTP response headers: HTTPHeaderDict({'Access-Control-All

___
#    